In [1]:
import gym
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate
import matplotlib as plt


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#wrapper for cont env with PPO, plot result for several steps
def ppo_eval_interval(p, L, t_t, n_iter, n_step, gae, learning_rate=0.0003):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
    cont_env = make_vec_env('inventory_cont_disc_action_config_fix_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1, 
                     learning_rate = learning_rate,use_sde = False, gae_lambda = gae, n_steps = n_step)
    env_eval = make_vec_env('inventory_cont_disc_action_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=60000)#each iteration has 8192 timesteps with n_env=4
        timesteps = timesteps + 32767+32767

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(-res_mean)
        res_std_arr.append(res_std)
        print(res_mean_arr)
    
#     plt.pyplot.plot(res_mean_arr)

    return res_mean_arr, res_std_arr

In [ ]:
listp = [0.25,1,4,9,39,99]
listL = [30,50]
# listp = [99]
# listL = [100]
t_t = 2000000
n_iter = 10000 #for eval
n_step = 8192
learning_rate = 0.0003
gae_lambda = 0.99
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        res_mean, res_std = ppo_eval_interval(p,L,t_t, n_iter, n_step, gae_lambda, learning_rate)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':min(res_mean), 'res_std': res_std[np.argmin(res_mean)]}, ignore_index=True)

Running PPO w/: p= 0.25 , L= 30
Using cpu device
------------------------------
| time/              |       |
|    fps             | 5778  |
|    iterations      | 1     |
|    time_elapsed    | 5     |
|    total_timesteps | 32768 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2746        |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.008939523 |
|    clip_fraction        | 0.0374      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.99       |
|    explained_variance   | 0.000382    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.29e+05    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00396    |
|    value_loss           | 1.44e+06    |
--------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 2436        |
|    iterations           | 2           |
|    time_elapsed         | 26          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.008926435 |
|    clip_fraction        | 0.0507      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.71       |
|    explained_variance   | 5.96e-07    |
|    learning_rate        | 0.0003      |
|    loss                 | 208         |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00321    |
|    value_loss           | 2.16e+03    |
-----------------------------------------
mean:  -0.6728493872814576
standard deviation: 0.028085074303073467
[3.4126274737773548, 1.6188173498322775, 1.1235614066466106, 0.8155912282923964, 0.7504986603449864, 0.6728493872814576]
-----------------------------------------
| time/      

-----------------------------------------
| time/                   |             |
|    fps                  | 2451        |
|    iterations           | 2           |
|    time_elapsed         | 26          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.007936189 |
|    clip_fraction        | 0.0871      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.28       |
|    explained_variance   | 0.0498      |
|    learning_rate        | 0.0003      |
|    loss                 | 19.7        |
|    n_updates            | 210         |
|    policy_gradient_loss | -0.00795    |
|    value_loss           | 62.2        |
-----------------------------------------
mean:  -0.3324931980147617
standard deviation: 0.004189413126771275
[3.4126274737773548, 1.6188173498322775, 1.1235614066466106, 0.8155912282923964, 0.7504986603449864, 0.6728493872814576, 0.541732245725311, 0.47043565176979846, 0.4160660329548

----------------------------------------
| time/                   |            |
|    fps                  | 6467       |
|    iterations           | 1          |
|    time_elapsed         | 5          |
|    total_timesteps      | 32768      |
| train/                  |            |
|    approx_kl            | 0.00838295 |
|    clip_fraction        | 0.0807     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.95      |
|    explained_variance   | 0.0569     |
|    learning_rate        | 0.0003     |
|    loss                 | 6.11       |
|    n_updates            | 300        |
|    policy_gradient_loss | -0.00568   |
|    value_loss           | 27.8       |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2773        |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 65536       |
| train/  

-----------------------------------------
| time/                   |             |
|    fps                  | 2800        |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.006701448 |
|    clip_fraction        | 0.0621      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.72       |
|    explained_variance   | 0.0136      |
|    learning_rate        | 0.0003      |
|    loss                 | 2           |
|    n_updates            | 390         |
|    policy_gradient_loss | -0.00327    |
|    value_loss           | 29.8        |
-----------------------------------------
mean:  -0.24074859483301364
standard deviation: 0.0028465521579328404
[3.4126274737773548, 1.6188173498322775, 1.1235614066466106, 0.8155912282923964, 0.7504986603449864, 0.6728493872814576, 0.541732245725311, 0.47043565176979846, 0.41606603295

...

-----------------------------------------
| time/                   |             |
|    fps                  | 2831        |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.008116659 |
|    clip_fraction        | 0.0329      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.32       |
|    explained_variance   | -4.77e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 30.3        |
|    n_updates            | 310         |
|    policy_gradient_loss | -0.000623   |
|    value_loss           | 1.9e+03     |
-----------------------------------------
mean:  -0.8191195604087561
standard deviation: 0.009468139619523936
[3.750866585619376, 1.932791645472187, 1.3606613799478573, 1.1924657183880865, 1.0559361744944624, 1.0096519411331903, 0.9843118580167804, 0.988251181956179, 0.9493584516063713

mean:  -0.8144289958030925
standard deviation: 0.008786569046205388
[3.750866585619376, 1.932791645472187, 1.3606613799478573, 1.1924657183880865, 1.0559361744944624, 1.0096519411331903, 0.9843118580167804, 0.988251181956179, 0.9493584516063713, 0.8996718408721629, 0.866218706800882, 0.8481586744586173, 0.8309368857070487, 0.8195641296966268, 0.8260421032860675, 0.8191195604087561, 0.8133318413549437, 0.8183456515887066, 0.8086004067421667, 0.8144289958030925]
-----------------------------------------
| time/                   |             |
|    fps                  | 6049        |
|    iterations           | 1           |
|    time_elapsed         | 5           |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.008238462 |
|    clip_fraction        | 0.0295      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.24       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0

-----------------------------------------
| time/                   |             |
|    fps                  | 6279        |
|    iterations           | 1           |
|    time_elapsed         | 5           |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.008116722 |
|    clip_fraction        | 0.0307      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.04       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 61          |
|    n_updates            | 480         |
|    policy_gradient_loss | -0.000686   |
|    value_loss           | 1.86e+03    |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2700        |
|    iterations           | 2           |
|    time_elapsed         | 24          |
|    total_timesteps      | 65536 

-----------------------------------------
| time/                   |             |
|    fps                  | 7007        |
|    iterations           | 1           |
|    time_elapsed         | 4           |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.009826578 |
|    clip_fraction        | 0.037       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.96       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 46.4        |
|    n_updates            | 560         |
|    policy_gradient_loss | -0.000636   |
|    value_loss           | 1.69e+03    |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2788        |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 65536 

-----------------------------------------
| time/                   |             |
|    fps                  | 2679        |
|    iterations           | 2           |
|    time_elapsed         | 24          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.008709198 |
|    clip_fraction        | 0.0588      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.95       |
|    explained_variance   | 3.06e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 5.35e+04    |
|    n_updates            | 30          |
|    policy_gradient_loss | -0.00542    |
|    value_loss           | 1.4e+05     |
-----------------------------------------
mean:  -1.8378250771335995
standard deviation: 0.1240278018370099
[3.7452422012793205, 1.8378250771335995]
-----------------------------------------
| time/                   |             |
|    fps                  | 5443        |
|    iterat

In [ ]:
ppo_res

In [ ]:
plt.pyplot.plot(res_mean)